# Kaggle Getting Started Prediction Competition: Store Sales - Time Series Forecasting

In this [competition](https://www.kaggle.com/competitions/store-sales-time-series-forecasting), we will use time-series forecasting to forecast store sales on data from Corporación Favorita, a large Ecuadorian-based grocery retailer. The notebook is a buildup of hands-on-exercises presented in Kaggle Learn course of [Time Series Course](https://www.kaggle.com/learn/time-series) where you will learn to leverage periodic trends for forecasting as well as combine different models such as linear regression and XGBoost to perfect your forecasting. For the purpose of this tutorial we are looking at periodic trend for forecasting.

## Install necessary packages

We can install the necessary package by either running `pip install --user <package_name>` or include everything in a `requirements.txt` file and run `pip install --user -r requirements.txt`. We have put the dependencies in a `requirements.txt` file so we will use the former method.

Restart the kernel after installation

In [1]:
# !pip install --user -r requirements.txt

In [2]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

In [3]:
path = 'data'
# path = os.getcwd()

train_data_filepath = path + "/train.csv"
test_data_filepath = path + "/test.csv"
holidays_filepath = path + "/holidays_events.csv"

# Read the csv files into dataframes
# Training data
train_sales = pd.read_csv(train_data_filepath,
    usecols=['store_nbr', 'family', 'date', 'sales'],
    dtype={
        'store_nbr': 'category',
        'family': 'category',
        'sales': 'float32',
    },
    parse_dates=['date'],
    infer_datetime_format=True,
)
train_sales['date'] = train_sales.date.dt.to_period('D')
train_sales = train_sales.set_index(['store_nbr', 'family', 'date']).sort_index()

# Holiday features dataset
holidays_events = pd.read_csv(
    holidays_filepath,
    dtype={
        'type': 'category',
        'locale': 'category',
        'locale_name': 'category',
        'description': 'category',
        'transferred': 'bool',
    },
    parse_dates=['date'],
    infer_datetime_format=True,
)
holidays_events = holidays_events.set_index('date').to_period('D')


# Test data id required for submission of forecast sales
df_test = pd.read_csv(
    test_data_filepath,
    dtype={
        'store_nbr': 'category',
        'family': 'category',
        'onpromotion': 'uint32',
    },
    parse_dates=['date'],
    infer_datetime_format=True,
)
df_test['date'] = df_test.date.dt.to_period('D')
df_test = df_test.set_index(['store_nbr', 'family', 'date']).sort_index()

In [4]:
train_sales.head()

sales
store_nbr family     date             
1         AUTOMOTIVE 2013-01-01    0.0
                     2013-01-02    2.0
                     2013-01-03    3.0
                     2013-01-04    3.0
                     2013-01-05    5.0

In [5]:
holidays_events.head()

,type,locale,locale_name,description,transferred
date,,,,,
2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


In [6]:
df_test.head()

id  onpromotion
store_nbr family     date                            
1         AUTOMOTIVE 2017-08-16  3000888            0
                     2017-08-17  3002670            0
                     2017-08-18  3004452            0
                     2017-08-19  3006234            0
                     2017-08-20  3008016            0

In [7]:
# National and regional holidays of Ecuador in the training set
# Holiday features
holidays = (
    holidays_events
    .query("locale in ['National', 'Regional']")
    .loc['2017':'2017-08-15', ['description']]
    .assign(description=lambda x: x.description.cat.remove_unused_categories())
)

In [8]:
print(holidays)

                                       description
date                                              
2017-01-01                      Primer dia del ano
2017-01-02             Traslado Primer dia del ano
2017-02-27                                Carnaval
2017-02-28                                Carnaval
2017-04-01           Provincializacion de Cotopaxi
2017-04-14                           Viernes Santo
2017-05-01                         Dia del Trabajo
2017-05-13                       Dia de la Madre-1
2017-05-14                         Dia de la Madre
2017-05-24                    Batalla de Pichincha
2017-05-26           Traslado Batalla de Pichincha
2017-06-25           Provincializacion de Imbabura
2017-08-10           Primer Grito de Independencia
2017-08-11  Traslado Primer Grito de Independencia


In [9]:
# Create training data features
y = train_sales.unstack(['store_nbr', 'family']).loc["2017"]

In [10]:
# Using CalendarFourier to create fourier features 
fourier = CalendarFourier(freq='M', order=4)

# Using DeterministicProcess to create indicators for both 
# weekly and monthly seasons
dp = DeterministicProcess(
    index=y.index,
    constant=True,
    order=1,
    seasonal=True,               # weekly seasonality (indicators)
    additional_terms=[fourier],  # annual seasonality (fourier)
    drop=True,
)

# `in_sample` creates features for the dates given in the `index` argument
X = dp.in_sample()

In [11]:
X.head()

,const,trend,"s(2,7)","s(3,7)","s(4,7)","s(5,7)","s(6,7)","s(7,7)","sin(1,freq=M)","cos(1,freq=M)","sin(2,freq=M)","cos(2,freq=M)","sin(3,freq=M)","cos(3,freq=M)","sin(4,freq=M)","cos(4,freq=M)"
date,,,,,,,,,,,,,,,,
2017-01-01,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000
2017-01-02,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.201299,0.979530,0.394356,0.918958,0.571268,0.820763,0.724793,0.688967
2017-01-03,1.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.394356,0.918958,0.724793,0.688967,0.937752,0.347305,0.998717,-0.050649
2017-01-04,1.0,4.0,0.0,0.0,1.0,0.0,0.0,0.0,0.571268,0.820763,0.937752,0.347305,0.968077,-0.250653,0.651372,-0.758758
2017-01-05,1.0,5.0,0.0,0.0,0.0,1.0,0.0,0.0,0.724793,0.688967,0.998717,-0.050649,0.651372,-0.758758,-0.101168,-0.994869


In [12]:
ohe = OneHotEncoder(sparse=False)

X_holidays = pd.DataFrame(
    ohe.fit_transform(holidays),
    index=holidays.index,
    columns=holidays.description.unique(),
)

X_holidays = pd.get_dummies(holidays)

# Join holiday features to training data
X_2= X.join(X_holidays, on='date').fillna(0.0)

In [13]:
X_2.head()

,const,trend,"s(2,7)","s(3,7)","s(4,7)","s(5,7)","s(6,7)","s(7,7)","sin(1,freq=M)","cos(1,freq=M)",...,description_Dia de la Madre-1,description_Dia del Trabajo,description_Primer Grito de Independencia,description_Primer dia del ano,description_Provincializacion de Cotopaxi,description_Provincializacion de Imbabura,description_Traslado Batalla de Pichincha,description_Traslado Primer Grito de Independencia,description_Traslado Primer dia del ano,description_Viernes Santo
date,,,,,,,,,,,,,,,,,,,,,
2017-01-01,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.000000,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-01-02,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.201299,0.979530,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2017-01-03,1.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.394356,0.918958,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-01-04,1.0,4.0,0.0,0.0,1.0,0.0,0.0,0.0,0.571268,0.820763,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-01-05,1.0,5.0,0.0,0.0,0.0,1.0,0.0,0.0,0.724793,0.688967,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
# Split the data to train and valid datasets
# X_train, X_valid, y_train, y_valid = train_test_split(X_2, y, test_size=0.2, shuffle=False)
# X_train, X_valid, y_train, y_valid = train_test_split(X_2, y, test_size=0.35, shuffle=False)
X_train, X_valid, y_train, y_valid = train_test_split(X_2, y, test_size=0.1, shuffle=False)

# Train the model
model = LinearRegression(fit_intercept=False)
model.fit(X_train, y_train)

LinearRegression(fit_intercept=False)

In [15]:
# Get the training and valid data predictions
y_train_pred = pd.DataFrame(model.predict(X_train), index=X_train.index, columns=y.columns)
y_valid_pred = pd.DataFrame(model.predict(X_valid), index=X_valid.index, columns=y.columns)
print(y_train_pred)
# Evaluate the model using mean_squared_log_error
print(y_valid_pred)

                   sales                                                      \
store_nbr              1                                                       
family        AUTOMOTIVE BABY CARE        BEAUTY     BEVERAGES         BOOKS   
date                                                                           
2017-01-01  7.549517e-15       0.0 -7.993606e-15  2.955858e-12 -7.771561e-16   
2017-01-02  5.000000e+00       0.0 -7.105427e-15  1.434000e+03 -2.220446e-16   
2017-01-03  3.909703e+00       0.0  4.124157e+00  2.618392e+03  1.045599e+00   
2017-01-04  4.126214e+00       0.0  3.838988e+00  2.712982e+03  1.081908e+00   
2017-01-05  3.775828e+00       0.0  2.772568e+00  2.300269e+03  7.568947e-01   
...                  ...       ...           ...           ...           ...   
2017-07-19  3.517707e+00       0.0  3.105532e+00  2.468518e+03  3.441017e-01   
2017-07-20  3.647447e+00       0.0  2.439251e+00  2.119398e+03  1.701704e-02   
2017-07-21  5.893711e+00       0.0  2.90

In [16]:
print(mean_absolute_error(y_valid, y_valid_pred))

77.5594297705251


In [17]:
# Create features for test set
# "out of sample" refers to times outside of the observation period of the training data.
# We are forecasting for next 16 days from the end of the training data date
test = dp.out_of_sample(steps=16)
test.index.name = 'date'
X_test = test.join(X_holidays, on='date').fillna(0.0)

In [18]:
X_test.head()

,const,trend,"s(2,7)","s(3,7)","s(4,7)","s(5,7)","s(6,7)","s(7,7)","sin(1,freq=M)","cos(1,freq=M)",...,description_Dia de la Madre-1,description_Dia del Trabajo,description_Primer Grito de Independencia,description_Primer dia del ano,description_Provincializacion de Cotopaxi,description_Provincializacion de Imbabura,description_Traslado Batalla de Pichincha,description_Traslado Primer Grito de Independencia,description_Traslado Primer dia del ano,description_Viernes Santo
date,,,,,,,,,,,,,,,,,,,,,
2017-08-16,1.0,228.0,0.0,0.0,1.0,0.0,0.0,0.0,0.101168,-0.994869,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-08-17,1.0,229.0,0.0,0.0,0.0,1.0,0.0,0.0,-0.101168,-0.994869,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-08-18,1.0,230.0,0.0,0.0,0.0,0.0,1.0,0.0,-0.299363,-0.954139,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-08-19,1.0,231.0,0.0,0.0,0.0,0.0,0.0,1.0,-0.485302,-0.874347,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-08-20,1.0,232.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.651372,-0.758758,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
y_forecast = pd.DataFrame(model.predict(X_test), index=X_test.index, columns=y.columns)
y_forecast

sales                                                          \
store_nbr           1                                                           
family     AUTOMOTIVE BABY CARE    BEAUTY    BEVERAGES     BOOKS BREAD/BAKERY   
date                                                                            
2017-08-16   4.530377       0.0  3.476643  2435.125539  0.539017   394.844494   
2017-08-17   4.243753       0.0  2.506978  2069.480159  0.146672   341.867130   
2017-08-18   5.801745       0.0  2.413335  2421.619056  0.123446   369.494607   
2017-08-19   4.404581       0.0  3.034082  2484.809788 -0.199899   380.191686   
2017-08-20   1.746713       0.0  1.132583  1047.939563 -0.242533   124.285728   
2017-08-21   4.197514       0.0  3.489445  2402.995314 -0.202355   380.765232   
2017-08-22   4.380181       0.0  3.885605  2373.957701  0.247319   338.210541   
2017-08-23   5.449226       0.0  4.377596  2482.942320  0.395875   401.550486   
2017-08-24   5.966248       0.0  3.532936  2078.429559  0.129517   338.248806   
2017-08-25   7.942797       0.0  3.253891  2359.660562  0.170662   355.829919   
2017-08-26   6.433461       0.0  3.414250  2332.943993 -0.166271   359.026534   
2017-08-27   3.159030       0.0  0.940934   811.966181 -0.285951   100.005596   
2017-08-28   4.693311       0.0  2.850934  2118.776864 -0.344358   358.929538   
2017-08-29   3.975547       0.0  3.129241  2102.279061  0.037795   324.592586   
2017-08-30   4.469587       0.0  3.900584  2293.759485  0.190055   400.848069   
2017-08-31   4.917283       0.0  3.620152  2028.937558  0.007847   352.649771   

                                                            ...            \
store_nbr                                                   ...         9   
family     CELEBRATION    CLEANING       DAIRY        DELI  ... MAGAZINES   
date                                                        ...             
2017-08-16   19.155803  837.879756  825.955214  140.726314  ...  3.273131   
2017-08-17   21.657030  668.495547  680.198257  118.434844  ...  2.869787   
2017-08-18   23.541928  752.379160  793.051857  180.492023  ...  3.098471   
2017-08-19   14.470116  619.148805  756.691792  142.406902  ...  4.445279   
2017-08-20    3.314559  218.241260  293.710673   61.056835  ...  5.587107   
2017-08-21   14.954285  671.859086  727.873511  145.608057  ...  2.926209   
2017-08-22   16.676904  769.524478  706.798103  134.702782  ...  2.673161   
2017-08-23   18.868402  866.401057  828.422416  135.273727  ...  2.636684   
2017-08-24   23.048885  689.752267  678.907893  109.559662  ...  2.113239   
2017-08-25   25.967477  756.719186  782.699559  169.236486  ...  2.071389   
2017-08-26   17.047643  597.937552  732.941236  130.398960  ...  3.199484   
2017-08-27    5.257965  169.218185  256.317846   50.524195  ...  4.358689   
2017-08-28   15.878498  603.429651  683.193370  139.224362  ...  2.033654   
2017-08-29   16.731948  701.063303  667.748359  135.095252  ...  2.375713   
2017-08-30   18.626602  823.291951  810.947465  144.203969  ...  3.011725   
2017-08-31   23.188303  694.554224  695.823441  127.045596  ...  3.002575   

                                                                           \
store_nbr                                                                   
family           MEATS PERSONAL CARE PET SUPPLIES PLAYERS AND ELECTRONICS   
date                                                                        
2017-08-16  315.692006    321.105247     8.063437                8.086183   
2017-08-17  515.972387    311.906195     6.568633                8.765447   
2017-08-18  295.035107    265.940697     7.409832                7.368460   
2017-08-19  413.255484    547.682569    10.337513               13.942843   
2017-08-20  422.096330    593.025760    10.615341               15.512749   
2017-08-21  320.496000    358.175192     5.904555                7.695611   
2017-08-22  294.686378    339.686169     5.467773                7.964100   
2017-08

In [20]:
y_submit = y_forecast.stack(['store_nbr', 'family'])
y_submit = y_submit.join(df_test.id).reindex(columns=['id', 'sales'])
y_submit.to_csv('submission.csv', index=False)